In [2]:
# 依存関係
'''
poetry add wheel

poetry add --group dev 'ipykernel' 'ipywidgets'

poetry add --group llm 'openvino'\
    "git+https://github.com/huggingface/optimum-intel.git"\
    "git+https://github.com/openvinotoolkit/nncf.git"
'''

'\npoetry add wheel\n\npoetry add --group dev \'ipykernel\' \'ipywidgets\'\n\npoetry add --group llm \'openvino\'    "git+https://github.com/huggingface/optimum-intel.git"    "git+https://github.com/openvinotoolkit/nncf.git"\n'

In [3]:
from pathlib import Path
import logging
import os
from datetime import datetime
from llm_config import LLM_MODELS_CONFIG

import openvino as ov
import nncf


INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, onnx, openvino


In [4]:
# ## login to huggingfacehub to get access to pretrained model 

# from huggingface_hub import notebook_login, whoami

# try:
#     whoami()
#     print('Authorization token already provided')
# except OSError:
#     notebook_login()

In [5]:
nncf.set_log_level(logging.ERROR)

# ダウンロードするモデル
company = "microsoft"
# company = "google"
# company = "rinna"

model_name = "Phi-3-mini-4k-instruct"
# model_name = "gemma-2b-it"
# model_name = "youri-7b-chat"

model_id = f'{company}/{model_name}'

remote_code = LLM_MODELS_CONFIG[model_name]['remote_code']  # Phi-3はTrue

# モデルを保存するディレクトリ
model_dir = Path(f'../../model/{model_name}')
fp16_model_dir = model_dir / "FP16"  # float 16bitモデルの保存先
int8_model_dir = model_dir / "INT8"  # 量子化モデルの保存先(8bit)
int4_model_dir = model_dir / "INT4"  # 量子化モデルの保存先(4bit)

In [6]:
remote_code

True

In [7]:
compression_configs = {
    # ここのパラメータは要調整
    # "sym":          対称量子化の利用
    # 'group_size':  グループサイズ  (64, 128が無難？)
    # 'ratio':       量子化後のパラメータの割合  (0.5~0.8で試す)
    "gemma-2b-it": {
        "sym": True,
        "group_size": 64,
        "ratio": 0.6,
    },
    "default": {
        "sym": False,
        "group_size": 128,
        "ratio": 0.8,
    },
}

In [8]:
model_id

'microsoft/Phi-3-mini-4k-instruct'

In [9]:
fp16_model_dir

PosixPath('../../model/Phi-3-mini-4k-instruct/FP16')

In [10]:
core = ov.Core()
# optimum-cliでモデルをopenvino形式でダウンロード
export_command_base = "optimum-cli export openvino --model {} --task text-generation-with-past".format(model_id)

def convert_to_fp16():
    global export_command_base
    export_command = ''
    # すでに存在する場合はスキップ
    if (fp16_model_dir / "openvino_model.xml" ).exists():
        return
    if remote_code:
        # Phi-3のみ
        export_command_base += " --trust-remote-code"
    export_command = export_command_base + " --weight-format fp16"
    export_command += " " + str(fp16_model_dir)
    # モデルのダウンロード開始時間
    start_model_download = datetime.now()
    print('export_command:', export_command)
    os.system(export_command)  # モデルのダウンロード
    # モデルのダウンロード終了時間
    end_model_download = datetime.now() - start_model_download
    print('export done', end_model_download.total_seconds())


def convert_to_int8():
    global export_command_base
    if (int8_model_dir / "openvino_model.xml").exists():
        return
    if remote_code:
        export_command_base += " --trust-remote-code"
    export_command = export_command_base + " --weight-format int8"
    export_command += " " + str(int8_model_dir)
    # モデルのダウンロード開始時間
    start_model_download = datetime.now()
    print('export_command:', export_command)
    os.system(export_command)  # モデルのダウンロード
    # モデルのダウンロード終了時間
    end_model_download = datetime.now() - start_model_download
    print('export done', end_model_download.total_seconds())

def convert_to_int4():
    global export_command_base
    if (int4_model_dir / "openvino_model.xml").exists():
        return
    if remote_code:
        export_command_base += " --trust-remote-code"
    # 量子化の設定
    model_compression_params  = compression_configs.get(model_name, compression_configs["default"])
    export_command = export_command_base + " --weight-format int4"
    int4_compression_args = " --group-size {} --ratio {}".format(model_compression_params["group_size"], model_compression_params["ratio"])
    if model_compression_params["sym"]:
        int4_compression_args += " --sym"
    export_command += int4_compression_args + " " + str(int4_model_dir)
    # モデルのダウンロード開始時間
    start_model_download = datetime.now()
    print('export_command:', export_command)
    os.system(export_command)  # モデルのダウンロード
    # モデルのダウンロード終了時間
    end_model_download = datetime.now() - start_model_download
    print('export done', end_model_download.total_seconds())


In [8]:
convert_to_int4()

export_command: optimum-cli export openvino --model microsoft/Phi-3-mini-4k-instruct --task text-generation-with-past --trust-remote-code --weight-format int4 --group-size 128 --ratio 0.8 ../../model/Phi-3-mini-4k-instruct/INT4
INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, onnx, openvino


/home/korryu/Documents/TECH.C/Taltner/server/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Framework not specified. Using pt to export the model.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.78s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/korryu/Documents/TECH.C/Taltner/server/.venv/lib/python3.10/site-packages/huggingfac

Mixed-Precision assignment ━━━━━━━━━━━━━━━━━━━━ 100% 128/128 • 0:01:14 • 0:00:0000:0200:06
INFO:nncf:Statistics of the bitwidth distribution:
┍━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┑
│   Num bits (N) │ % all parameters (layers)   │ % ratio-defining parameters (layers)   │
┝━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┥
│              8 │ 25% (27 / 130)              │ 21% (25 / 128)                         │
├────────────────┼─────────────────────────────┼────────────────────────���───────────────┤
│              4 │ 75% (103 / 130)             │ 79% (103 / 128)                        │
┕━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┙
Applying Weight Compression ━━━━━━━╺━━━━━━━━━━━━  36% 47/130 • 0:00:25 • 0:00:37export done 194.37537


Killed


In [ ]:
convert_to_int8()

export_command: optimum-cli export openvino --model google/gemma-2b-it --task text-generation-with-past --weight-format int8 ../../model/gemma-2b-it/INT8
INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, onnx, openvino


/home/korryu/Documents/TECH.C/Taltner/server/.venv/lib/python3.10/site-packages/transformers/utils/import_utils.py:521: FutureWarning: `is_torch_tpu_available` is deprecated and will be removed in 4.41.0. Please use the `is_torch_xla_available` instead.
  warnings.warn(
Framework not specified. Using pt to export the model.
/home/korryu/Documents/TECH.C/Taltner/server/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint sh

INFO:nncf:Statistics of the bitwidth distribution:
┍━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┑
│   Num bits (N) │ % all parameters (layers)   │ % ratio-defining parameters (layers)   │
┝━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┥
│              8 │ 100% (127 / 127)            │ 100% (127 / 127)                       │
┕━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━���━━━━━━━━━━━━━━━┙
Applying Weight Compression ━━━━━━━━━━━━━━━━━━━ 100% 127/127 • 0:00:26 • 0:00:0000:0100:02
export done


In [31]:
convert_to_fp16()

export_command: optimum-cli export openvino --model microsoft/Phi-3-mini-4k-instruct --task text-generation-with-past --trust-remote-code --weight-format fp16 ../../model/Phi-3-mini-4k-instruct/FP16
INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, onnx, openvino


/home/korryu/Documents/TECH.C/Taltner/server/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Framework not specified. Using pt to export the model.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards:  50%|█████     | 1/2 [00:03<00:03,  3.87s/it]

export done 15.817334


Killed


In [12]:
# モデルが保存されているディレクトリのサイズを確認

fp16_weights = fp16_model_dir / "openvino_model.bin"
int8_weights = int8_model_dir / "openvino_model.bin"
int4_weights = int4_model_dir / "openvino_model.bin"

if fp16_weights.exists():
    print(f"Size of FP16 model is {fp16_weights.stat().st_size / 1024 / 1024:.2f} MB")

for precision, compressed_weights in zip([8, 4], [int8_weights, int4_weights]):
    if compressed_weights.exists():
        print(f"Size of model with INT{precision} compressed weights is {compressed_weights.stat().st_size / 1024 / 1024:.2f} MB")
    if compressed_weights.exists() and fp16_weights.exists():
        print(f"Compression rate for INT{precision} model: {fp16_weights.stat().st_size / compressed_weights.stat().st_size:.3f}")

Size of FP16 model is 7288.13 MB
Size of model with INT4 compressed weights is 2336.74 MB
Compression rate for INT4 model: 3.119


In [13]:
# デバイスの選択
support_devices = core.available_devices
device = 'NPU' if 'NPU' in support_devices else 'CPU'  # 実機のNPUが使えればいいのだけれど。。。

In [14]:
device

'CPU'

In [15]:
from ipywidgets import widgets

available_models = []
if int4_model_dir.exists():
    available_models.append("INT4")
if int8_model_dir.exists():
    available_models.append("INT8")
if fp16_model_dir.exists():
    available_models.append("FP16")

model_to_run = widgets.Dropdown(
    options=available_models,
    value=available_models[0],
    description="Model to run:",
    disabled=False,
)

model_to_run

Dropdown(description='Model to run:', options=('INT4', 'FP16'), value='INT4')

In [16]:
model_to_run.value

'INT4'

In [17]:
from transformers import AutoConfig, AutoTokenizer
from optimum.intel.openvino import OVModelForCausalLM

if model_to_run.value == "INT4":  # 4bitモデルを使う場合
    model_dir = int4_model_dir
elif model_to_run.value == "INT8":  # 8bitモデルを使う場合
    model_dir = int8_model_dir
else:
    model_dir = fp16_model_dir  # 16bitモデルを使う場合
print(f"Loading model from {model_dir}")

ov_config = {"PERFORMANCE_HINT": "LATENCY", "NUM_STREAMS": "1", "CACHE_DIR": ""}

tok = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)

ov_model = OVModelForCausalLM.from_pretrained(
    model_dir,
    device=device,
    ov_config=ov_config,
    config=AutoConfig.from_pretrained(model_dir, trust_remote_code=True),
    trust_remote_code=True,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading model from ../../model/Phi-3-mini-4k-instruct/INT4


/home/korryu/Documents/TECH.C/Taltner/server/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The argument `trust_remote_code` is to be used along with export=True. It will be ignored.
Compiling the model to CPU ...


In [24]:
# モデルの動作確認
tokenizer_kwargs = LLM_MODELS_CONFIG[model_name].get("tokenizer_kwargs", {})

SYSTEM_PROMPT = '''\
# 指示
あなたは、ユーザーが初対面の相手との会話を補助するためのアシスタントです。
ユーザーと相手の会話を円滑に進めるために、ユーザーの補佐をします。
以下の要素を考慮して、4つの話題をJSONフォーマットで提示しなさい。

# 条件
1. 初対面の相手との会話を始めるのに適した話題の提供
2. 会話の流れを考慮して、次の話題を提示
3. 単語くらいの短い文章で話題を提示
4. 出力フォーマットは{{JSON}}形式

以下のフォーマットで出力してください：

```json
{
  "topics": [
    "話題1",
    "話題2",
    "話題3",
    "話題4"
  ]
}
```

# 例1
```json
{
  "topics": [
    "好きな食べ物",
    "どこ出身",
    "好きなゲーム",
    "最近見た映画"
  ]
}
```

# 例2
```json
{
  "topics": [
    "趣味",
    "休日の過ごし方",
    "好きな音楽",
    "行ってみたい場所"
  ]
}
```

# 例3
```json
{
  "topics": [
    "好きなスポーツ",
    "好きなアニメ",
    "好きな漫画",
    "好きな映画"
  ]
}
```\
'''

conversation = '''\
自分: 「初めまして〜。俺AI専攻の人と話すの初めてなんだけど”AI専攻ってどんなことしてるの”？」

相手: 「AI専攻は機械学習とか画像や音声認識とかLLMについて勉強してるよ」
'''

start_message: str = LLM_MODELS_CONFIG[model_name]['start_message']
prompt_template: str = LLM_MODELS_CONFIG[model_name]['prompt_template']

sys_prompt = start_message.format(
    SYSTEM_PROMPT=SYSTEM_PROMPT
)

prompt = sys_prompt + prompt_template.format(
    user=conversation
)

input_tokens = tok(prompt, return_tensors="pt", **tokenizer_kwargs)

answer = ov_model.generate(
    **input_tokens,
    max_new_tokens=100,
    temperature=0.7,
    do_sample=True
)

ans = tok.batch_decode(answer, skip_special_tokens=True)[0]
print(ans)

# 指示
あなたは、ユーザーが初対面の相手との会話を補助するためのアシスタントです。
ユーザーと相手の会話を円滑に進めるために、ユーザーの補佐をします。
以下の要素を考慮して、4つの話題をJSONフォーマットで提示しなさい。

# 条件
1. 初対面の相手との会話を始めるのに適した話題の提供
2. 会話の流れを考慮して、次の話題を提示
3. 単語くらいの短い文章で話題を提示
4. 出力フォーマットは{{JSON}}形式

以下のフォーマットで出力してください：

```json
{
  "topics": [
    "話題1",
    "話題2",
    "話題3",
    "話題4"
  ]
}
```

# 例1
```json
{
  "topics": [
    "好きな食べ物",
    "どこ出身",
    "好きなゲーム",
    "最近見た映画"
  ]
}
```

# 例2
```json
{
  "topics": [
    "趣味",
    "休日の過ごし方",
    "好きな音楽",
    "行ってみたい場所"
  ]
}
```

# 例3
```json
{
  "topics": [
    "好きなスポーツ",
    "好きなアニメ",
    "好きな漫画",
    "好きな映画"
  ]
}
``` 自分: 「初めまして〜。俺AI専攻の人と話すの初めてなんだけど”AI専攻ってどんなことしてるの”？」

相手: 「AI専攻は機械学習とか画像や音声認識とかLLMについて勉強してるよ」
 ```json
{
  "topics": [
    "AIにおける最近の進展",
    "機械学習の基本的な概念",

    "画像認識の応用例",

    "音声認識の革新的な進展"
  ]
}
```
